In [ ]:
lastlevel_shp = '/home/jasp/autocls_exp3/pxs/px_sample_5.shp'

ref_rst = '/home/jasp/autocls_exp3/pxs_rst/refpxgroups_06x06.tif'

pk = 'cid'

lastlevel_rst = '/home/jasp/autocls_exp3/pxs_rst/refpxgroups_06x06_5.tif'

lulcmaps = {
    'clcp2018' : '/mnt/d/autocls_exp3/lulc_06x06/clcpp_bb2018_06x06.tif',
    'cos2018'  : '/mnt/d/autocls_exp3/lulc_06x06/cos_2018_06x06.tif',
    'cosc2021' : '/mnt/d/autocls_exp3/lulc_06x06/cosc_2021_06x06.tif',
    'ua2018'   : '/mnt/d/autocls_exp3/lulc_06x06/ua_2018_06x06.tif'
}

lulc_col = {
    'clcp2018' : 'Value',
    'cos2018'  : 'COS18n4_C',
    'cosc2021' : 'Value',
    'ua2018'   : 'code_2018'
}

lulc_mixed = {
    'clcp2018' : '/home/jasp/autocls_exp3/px_mixed/mixed4_clcp2018.tif',
    'cos2018'  : '/home/jasp/autocls_exp3/px_mixed/mixed4_cos2018.tif',
    'cosc2021' : '/home/jasp/autocls_exp3/px_mixed/mixed4_cosc2021.tif',
    'ua2018'   : '/home/jasp/autocls_exp3/px_mixed/mixed4_ua2018.tif'
}

pure_px_tbl = '/home/jasp/autocls_exp3/px_classes'
pure_bname = 'pxclass5'

db = 'gthruth_valeflores_v3'

In [ ]:
import os

from glass.dtt.rst.torst import shp_to_rst
from glass.pys.tm import now_as_str
from glass.wenv.grs import run_grass
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.rst.alg import grsrstcalc
from glass.rst.local.grs import combine_report_to_df, grs_combine
from glass.wt import obj_to_tbl
from glass.prop.prj import get_epsg
from glass.sql.q import q_to_obj
from glass.rd import tbl_to_obj

In [ ]:
epsg = get_epsg(ref_rst)

llrst = shp_to_rst(
    lastlevel_shp, pk, None, 0,
    lastlevel_rst,
    rst_template=ref_rst, api="gdal",
    rtype=int, dtype="UInt32"
)

In [ ]:
# Get Lulc Raster Tables

lulc_rst_tbls = {}

for m in lulc_col:
    lulc_rst_tbls[m] = q_to_obj(db, (
        "SELECT mtbl.classid, mtbl.code, mtbl.classname, jtbl.slug "
        "FROM mapclasses AS mtbl "
        "LEFT JOIN refmaps AS jtbl "
        "ON mtbl.mapfk = jtbl.mapid "
        f"WHERE jtbl.slug = '{m}'"
    ))
    
    if lulc_col[m] != 'Value':
        _df = tbl_to_obj(f'{lulcmaps[m]}.vat.dbf')

        lulc_rst_tbls[m] = lulc_rst_tbls[m].merge(
            _df, how='right', left_on='code',
            right_on=lulc_col[m]
        )

        lulc_rst_tbls[m].drop(['Count', lulc_col[m]], axis=1,  inplace=True)
        lulc_rst_tbls[m].rename(columns={'Value' : 'rvalue'}, inplace=True)

    else:
        lulc_rst_tbls[m]['rvalue'] = lulc_rst_tbls[m]['code'].astype(int)

In [ ]:
# Create GRASS GIS Session
loc = now_as_str(utc=True)
ws = pure_px_tbl

gb = run_grass(ws, location=loc, srs=ref_rst)

import grass.script.setup as gsetup

gsetup.init(gb, ws, loc, 'PERMANENT')

In [ ]:
# Import Raster's

grsll = rst_to_grs(lastlevel_rst, as_cmd=True)

grslulc = [rst_to_grs(
    lulcmaps[r], grst=r, as_cmd=True
) for r in lulcmaps]

for m in lulc_mixed:
    lulc_mixed[m] = rst_to_grs(lulc_mixed[m], as_cmd=True)

In [ ]:
# Get final relations
for r in grslulc:
    # Delete mixed pixels from sample fishnet
    _grsll = grsrstcalc(
        f"{grsll} * {lulc_mixed[r]}",
        f"{grsll}_{r}",
        ascmd=True
    )

    cmbrst = grs_combine([_grsll, r], f'cmb_{r}')

    cmb_table = combine_report_to_df(
        cmbrst, ['pxid', 'lulc_class_value'],
        otxt=os.path.join(ws, loc, f'{cmbrst}.txt')
    )

    cmb_table = cmb_table.merge(
        lulc_rst_tbls[r], how='left',
        left_on='lulc_class_value',
        right_on='rvalue'
    )

    _cmb_tbl = cmb_table[['pxid', 'classid']]

    _cmb_tbl.rename(columns={'classid' : 'clsid'}, inplace=True)

    # Export table
    obj_to_tbl(_cmb_tbl, os.path.join(pure_px_tbl, f"{pure_bname}_{r}.xlsx"))